# "시계열 데이터 Segmentation"
> "Vibration data"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [coding, jupyter]

# 시계열 데이터 Segmentation & Data Loader 만들기
- 아래의 코드는 2021 PHMAP Data Challenge Regression Task에서 사용한  Data Loader의 코드임


In [ ]:
## csv 파일 이름 불러오기
file_name = glob.glob('C:/Users/dldls/Desktop/따오기/210708_New/raw_data/*.csv')

In [ ]:
# csv 파일 읽고, 전처리해서 원하는 값들을 data_set 리스트에 추가
data_set = []
for i in range(len(file_name)):
    data = pd.read_csv(file_name[i])
    data = np.concatenate([np.array(data)[:,1].reshape(len(data),1),np.array(data)[:,2].reshape(len(data),1)],axis=1)
    data = data.transpose(1,0)
    data_set.append(np.array(data))

- 아래 코드는 긴 시계열 데이터를 10등분하고 , 10등분한 data를 다시 10등분해 , 앞의 9개는 학습 데이터, 뒤의 1개는 테스트 데이터로 사용
- 위와 같이 학습/테스트 데이터를 나눈 이유는 한 개의 파일이라도 구간에 따라서 진동 데이터의 경향이 매우 다르기 때문.

In [ ]:
def devide_data(data) : 
    split = np.array_split(data,10) # split = data 10등분
    for i in range(10) : 
        split_split = np.array_split(split[i],10) # split split은 10등분한 데이터를 다시 10등분해서 
        # 앞 9개는 train
        for j in range(9) : 
            if j == 0 : 
                tr = split_split[0]
            else : 
                tr=np.hstack([tr,split_split[j]])
        # 뒤 1개는 validation 
        val = np.hstack([split_split[-1]])
        if i ==0 : 
            train = tr
            valid = val
        else : 
            train = np.hstack([train,tr])
            valid = np.hstack([valid,val])
    return train, valid

- 아래는 각 파일별로 train / validation 데이터를 나눈 후, 각각 저장하는 코드 


In [ ]:
train_motor = []
valid_motor = []
train_skrew = []
valid_skrew = []
for i in range(len(file_name)): 
    data_motor = data_set[i][0]
    data_skrew = data_set[i][1]
    tr_motor , val_motor = devide_data(data_motor)
    tr_skrew , val_skrew = devide_data(data_skrew)
    if i == 0 : 
        train_motor = tr_motor
        valid_motor = val_motor
        train_skrew = tr_skrew
        valid_skrew = val_skrew
    else : 
        train_motor = np.hstack([train_motor,tr_motor])
        valid_motor = np.hstack([valid_motor,val_motor])
        train_skrew = np.hstack([train_skrew,tr_skrew])
        valid_skrew = np.hstack([valid_skrew,val_skrew])

- 앞에서 잘 자른 학습/테스트 데이터를 이용해 Pytorch model에서 사용 할 Data Loader를 만드는 코드 
- 여기서는 긴 시퀀스의 시계열 데이터를 15개 간격으로 잘라서 각각의 데이터를 형성

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,data_motor,data_skrew):
        self.data_motor=data_motor
        self.data_skrew=data_skrew
    def __len__(self):
        return (len(self.data_motor)-4216)//15
    def __getitem__(self,idx):
        x1=torch.FloatTensor(self.data_motor[idx*15:idx*15+3162]).view([1,3162]) # motor
        x2=torch.FloatTensor(self.data_skrew[idx*15:idx*15+3162]).view([1,3162]) # skrew
        x3=torch.FloatTensor(self.data_motor[idx*15+3162:idx*15+4216]).view([1,1054]) # motor
        label=torch.FloatTensor(self.data_skrew[idx*15+3162:idx*15+4216]).view([1,1054]) # skrew
        return x1,x2,x3,label


- train과 valid data를 이용해서 각각 train_loader 와 val_loader를 형성 

In [ ]:
train_set=CustomDataset(train_motor,train_skrew)
val_set=CustomDataset(valid_motor,valid_skrew)
train_loader = DataLoader(train_set,batch_size = 32, shuffle=True)
val_loader = DataLoader(val_set,batch_size = 32, shuffle=True)